In [1]:
import pandas as pd 

In [20]:
metadata = pd.read_csv("metadata.csv")
cols = list(metadata["var_name"])
cols = [c.lower() for c in cols]
metadata["name_cleaned"] = cols
metadata.head(2)

,var_name,desc,name_cleaned
0,Year,1987-2008,year
1,Month,1-12,month


In [24]:
cols = []
for r in metadata.iterrows():
    row = r[1]
    this_col = {}
    this_col["name"] = row["name_cleaned"]
    this_col["description"] = row["desc"]
    this_col["type"] = "character"
    cols.append(this_col)
    
new_col = {}
new_col["name"] = "file"
new_col["description"] = "name of source file"
new_col["type"] = "character"
cols.append(new_col)

In [35]:
meta = {}
meta["id"] = "flights.flightsdata"
meta["table_desc"] = "Flights data from http://stat-computing.org/dataexpo/2009/the-data.html"
meta["table_name"] = "flightsdata"
meta["data_format"] = "csv_quoted_nodate"
meta["location"] = "s3://robintest-gluenotebooks3files/flights"
meta["glue_specific"] = {}
meta["glue_specific"]["PartitionKeys"] = [{"Name":"file", "Type":"string"}]
meta["columns"] = cols

In [36]:
import json
from dataengineeringutils.utils import write_json
write_json(meta, "metadata/metadata.json")

In [47]:
from dataengineeringutils.glue import metadata_folder_to_database
metadata_folder_to_database("metadata")

In [48]:
from dataengineeringutils.athena import make_partitions
temp_dir = 's3://robintest-gluenotebooks3files/glue_temp_dir/'
make_partitions("flights", "flightsdata", temp_dir)